## Tutorial to perform hyperparameter search for DeepKoopman on NACA0012 data
Do this tutorial after [`run.ipynb`](./run.ipynb).

You might be wondering how we arrived at the input settings for the 'Optimizing' section. The method `deepk.hyp_search:run_hyp_search()` performs hyperparameter search by sweeping inputs to `DeepKoopman` and collecting the loss and ANAE values. These can then be used to select 'good' input settings.

In [1]:
from deepk.hyp_search import run_hyp_search

### Load data

In [2]:
import pickle
with open('./data.pkl', 'rb') as f:
    data = pickle.load(f)

In [3]:
run_hyp_search(
    data = data,
    hyp_options = {
        'rank': [3,6,8,10],
        'encoded_size': [200,500,1000],
        'encoder_hidden_layers': [[500,500],[500,1000],[1000,500],[500,500,500]],
        'numepochs': 500,
        'clip_grad_norm': [None,5.,10.],
        'clip_grad_value': [None,2.]
    },
    numruns = 2
)

  0%|          | 0/2 [00:00<?, ?it/s]

********************************************************************************
********************************************************************************
Starting DeepKoopman hyperparameter search. Results will be stored in /Users/sourya/work/Essence/deep-koopman/examples/naca0012/hyp_search_GEnnhVHRNjkehezgnpAsLT.csv.

Performing total 2 runs. You can interrupt the script at any time (e.g. Ctrl+C), and intermediate results will be available in the above file.

Hyperparameters swept over:
rank : 3, 6, 8, 10
encoded_size : 200, 500, 1000
encoder_hidden_layers : [500, 500], [500, 1000], [1000, 500], [500, 500, 500]
clip_grad_norm : None, 5.0, 10.0
clip_grad_value : None, 2.0

Constant hyperparameters:
numepochs : 500

Other hyperparameters have default values, see https://galoisinc.github.io/deep-koopman/core.html
********************************************************************************
********************************************************************************

Hyper

 50%|█████     | 1/2 [00:13<00:13, 13.66s/it]




Hyperparameters: {'rank': 3, 'encoded_size': 500, 'encoder_hidden_layers': [500, 500, 500], 'clip_grad_norm': None, 'clip_grad_value': None}
Deep Koopman log file = /Users/sourya/work/Essence/deep-koopman/examples/naca0012/dk_HSeN34BmwuZ5mtwsijPMfS.log


100%|██████████| 2/2 [00:28<00:00, 14.40s/it]

PosixPath('/Users/sourya/work/Essence/deep-koopman/examples/naca0012/hyp_search_GEnnhVHRNjkehezgnpAsLT.csv')

We highly recommend performing hyperparameter search for any problem as it can lead to massively improved results and overcome numerical instabilities. If required, increase `numruns` to several hundred or even several thousand, which can take several hours to run, but the results are usually worth it.